In [1]:
# import the necessary libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load data
df_transformed = pd.read_csv(".data/processed/data-population-victims-of-armed-conflict.csv")

# Analize df
df_transformed.head(5)

# Load

## Creation of DW

In [ ]:
# Create database for DW
conn_dw = sqlite3.connect(ruta_base + "Project_ETL.db")
cursor_dw = conn_dw.cursor()

cursor_dw.executescript("""

-- DESACTIVAR CLAVES FORÁNEAS
PRAGMA foreign_keys = OFF;


-- ELIMINAR TABLAS (orden correcto)
DROP TABLE IF EXISTS victims;
DROP TABLE IF EXISTS person;
DROP TABLE IF EXISTS victimizing_act;
DROP TABLE IF EXISTS location;
DROP TABLE IF EXISTS registration_date;

-- ACTIVAR CLAVES FORÁNEAS
PRAGMA foreign_keys = ON;


-- CREAR DIMENSIONES
CREATE TABLE person (
    id_person INTEGER PRIMARY KEY NOT NULL,
    sex TEXT CHECK (sex IN ('Masculino', 'Femenino')),
    ethnic_group TEXT,
    classification TEXT
);

CREATE TABLE victimizing_act (
    id_act INTEGER PRIMARY KEY NOT NULL,
    victimization_fact TEXT
);

CREATE TABLE location (
    id_location INTEGER PRIMARY KEY NOT NULL,
    commune TEXT
);

CREATE TABLE registration_date (
    id_date INTEGER PRIMARY KEY NOT NULL,
    date_processing DATE,
    year INTEGER
);

CREATE TABLE victims (
    id_person INTEGER,
    id_act INTEGER,
    id_location INTEGER,
    id_date INTEGER,
    vulnerability_index INTEGER,
    total_victim INTEGER,
    FOREIGN KEY (id_person) REFERENCES person(id_person),
    FOREIGN KEY (id_act) REFERENCES victimizing_act(id_act),
    FOREIGN KEY (id_location) REFERENCES location(id_location),
    FOREIGN KEY (id_date) REFERENCES registration_date(id_date)
);""")

## separate column for data entry

In [ ]:
# person column data
people = df_transformed[["sex", "ethnic_group","classification"]].drop_duplicates()
people["id_person"] = people.index + 1
people.head(5)

In [ ]:
# victimizing_act column data
victimizing_act = df_transformed[["victimization_fact"]].drop_duplicates().reset_index(drop=True)
victimizing_act["id_act"] = victimizing_act.index + 1
victimizing_act.head(15)

In [ ]:
# registration_date column data
registration_date = df_transformed[["date_processing"]].drop_duplicates().copy()
registration_date["date_processing"] = pd.to_datetime(registration_date["date_processing"])
registration_date["year"] = registration_date["date_processing"].dt.year
registration_date["id_date"] = registration_date.index + 1
registration_date.head(10)

In [ ]:
# location column data
location =  df_transformed[["commune"]].drop_duplicates()
location["id_location"] = location.index + 1
location.head(23)

In [ ]:
# Load data into the DW
people.to_sql("person", conn_dw, if_exists="append", index=False)
victimizing_act.to_sql("victimizing_act", conn_dw, if_exists="append", index=False)
location.to_sql("location", conn_dw, if_exists="append", index=False)
registration_date.to_sql("registration_date", conn_dw, if_exists="append", index=False)

In [ ]:
# Function to display a table
def display_table(name_table):
  df_table = pd.read_sql(f"SELECT * FROM {name_table}", conn_dw)
  print(f"Tabla: {name_table}")
  display(df_table)


# Visualizar cada tabla del modelo dimensional
display_table("person")
display_table("victimizing_act")
display_table("location")
display_table("registration_date")

In [ ]:
# Copiar dataframe original
fact = df_transformed.copy()

#  The two columns must have the same type in both dataframes
fact["date_processing"] = pd.to_datetime(fact["date_processing"])
registration_date["date_processing"] = pd.to_datetime(registration_date["date_processing"])


# Merge with person
fact = fact.merge(
    people[["id_person","sex","ethnic_group","classification"]],
    on=["sex","ethnic_group","classification"],
    how="left"
)


# Merge with victimizing_act
fact = fact.merge(
    victimizing_act[["id_act","victimization_fact"]],
    on="victimization_fact",
    how="left"
)


# Merge with location
fact = fact.merge(
    location[["id_location","commune"]],
    on="commune",
    how="left"
)


# Merge with date
fact = fact.merge(
    registration_date[["id_date","date_processing"]],
    on="date_processing",
    how="left"
)

# remove descriptive columns
fact = fact.drop(columns=[
    "sex",
    "ethnic_group",
    "vulnerability_level",
    "victimization_fact",
    "commune",
    "date_processing"
])

In [ ]:
# Copiar dataframe original
fact = df_transformed.copy()

#  The two columns must have the same type in both dataframes
fact["date_processing"] = pd.to_datetime(fact["date_processing"])
registration_date["date_processing"] = pd.to_datetime(registration_date["date_processing"])


# Merge with person
fact = fact.merge(
    people[["id_person","sex","ethnic_group","classification"]],
    on=["sex","ethnic_group","classification"],
    how="left"
)


# Merge with victimizing_act
fact = fact.merge(
    victimizing_act[["id_act","victimization_fact"]],
    on="victimization_fact",
    how="left"
)


# Merge with location
fact = fact.merge(
    location[["id_location","commune"]],
    on="commune",
    how="left"
)


# Merge with date
fact = fact.merge(
    registration_date[["id_date","date_processing"]],
    on="date_processing",
    how="left"
)

# remove descriptive columns
fact = fact.drop(columns=[
    "sex",
    "ethnic_group",
    "vulnerability_level",
    "victimization_fact",
    "commune",
    "date_processing"
])

In [ ]:
# Load data to fact table
fact_table.to_sql("victims", conn_dw, if_exists="append", index=False)

In [ ]:
display_table("victims")